# LoC proportional thresholding and quantification notebook

This notebook loads a set of Z-tracks taken across an image volume from the lung-on-a-chip project and thresholds positive/negative expression of set markers based on a manual and unbiased assessment of the underlying images:

In [16]:
import btrack # import tracking software
import napari # image viewer
from skimage import io # image loader and viewer
import os # for filename manipulations
import glob # for more fn operations 
import pandas as pd # module for data org 
import matplotlib.pyplot as plt # module for plotting
import seaborn as sns # module for plotting
import numpy as np # for math operations
import re
from tqdm.auto import tqdm
from homuncu_loc import dataio
sns.set( style = 'white') # my favourite formatting :)
sns.set_palette('PiYG') # my favourite colours :D 

## 1. Load images and labels

In [7]:
pct_threshold = 0.3

In [ ]:
root_dir = '/Volumes/lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis' # /mnt/NEMO/home/shared/Lung on Chip/homuncu_loc_image_analysis

# root_dir = '/proj-lm-gutierrez/homuncu_loc_image_analysis/
markers = ['NKX21', 'PDPN', 'SPC']

sc_fn_pattern = os.path.join(root_dir, f'iAT1_iAT2_experiments/*/sc_analyses/*iat2.h5')
sc_fns = glob.glob(sc_fn_pattern)


for sc_fn in tqdm(sc_fns, total = len(sc_fns)):
    sc_fn_basename = os.path.basename(sc_fn)
    sc_base_dir = os.path.dirname(sc_fn)
    output_dir = os.path.join(sc_base_dir, 'proportional_thresholding')
    os.makedirs(output_dir, exist_ok = True)
    os.makedirs('temp_output', exist_ok = True)
    # load labels (tracks and segmentation)
    with btrack.io.HDF5FileHandler(sc_fn, 
                                       'r', 
                                           obj_type='obj_type_2'#'obj_type_2'
                                       ) as reader:
            # tracks = [t for t in reader.tracks if np.mean(t.properties['area']) > 1000]
            tracks = [t for t in reader.tracks if len(t) >= 3] #reader.tracks#
            segmentation = reader.segmentation
    napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim = 2)
    recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks)

    for channel, marker in enumerate(sc_fn.split('/')[-3].split('_')):  
        if marker not in markers:
            continue
        
        sc_intensities = [np.mean(track.properties['mean_intensity'][:,channel]) for track in tracks]
        max_sc_intensity = np.nanmax(sc_intensities)
        min_sc_intensity = np.nanmin(sc_intensities)
        threshold = int((pct_threshold * (max_sc_intensity - min_sc_intensity)) + min_sc_intensity)
    
        n_positive = len([t for t in tracks if np.max(t.properties['mean_intensity'][:,channel]) >= threshold])
        n_negative = len([t for t in tracks if np.max(t.properties['mean_intensity'][:,channel]) < threshold])
        
        print('\n', sc_fn_basename) 
        print(f'Number of channel {channel} positive cells: {n_positive}')
        print(f'Number of channel {channel} negative cells: {n_negative}')
        percentage = n_positive / (n_positive+n_negative) *100
        print(f'Percentage of positive cells: {percentage:.2f}%')

        plt.clf()
        # sc_intensities = [np.max(t.properties['mean_intensity'][:,channel]) for t in tracks]
        ax = sns.histplot(sc_intensities)
        plt.vlines(x=threshold,
               ymax=ax.get_ylim()[-1],
               ymin=ax.get_ylim()[0],
               color=sns.color_palette('PiYG')[-1],  # my favorite colors :D
               linewidth=1)
        sns.despine(offset = 10)
        plt.xlabel(f'Single-cell max intensity pixel value for channel {channel}, {marker}')
        plt.ylabel('Number of cells')
        plt.legend([f'Threshold ({threshold})'], loc='lower right')
        ax.text(0.4, 0.95, 
            f'Number of channel {channel} positive cells: {n_positive}\nNumber of channel {channel} negative cells: {n_negative}\nPercentage of positive cells: {percentage:.2f}%',
            transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=dict(boxstyle='round,pad=0.5', facecolor='grey', edgecolor='none', alpha=0.5))
        
        # image_name = os.path.basename(image_fn).replace('.tif','')
        image_ID = os.path.basename(sc_fn).replace('_iat2.h5','')
        
        plt.title(f'Distribution of single-cell maximum pixel value for channel {channel}\n image {image_ID}')
        output_fn = os.path.join(output_dir, f'{image_ID}_ch{channel}_proportional_threshold_quantification.png')
        plt.savefig(output_fn, bbox_inches = 'tight', dpi = 314)
        plt.savefig(f'temp_output/{image_ID}_ch{channel}_proportional_threshold_quantification.png', bbox_inches = 'tight', dpi = 314)

        # plt.show()


  0%|          | 0/22 [00:00<?, ?it/s]

[INFO][2024/04/29 03:30:06 PM] Opening HDF file: /Volumes/lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_SPC_PDPN_ZO1/sc_analyses/20230801_20X_23-03-011B6_DAPI_SP-C_PDPN_ZO-1_Multichannel Z-Stack_20230801_1440_iat2.h5...
[INFO][2024/04/29 03:30:06 PM] Loading tracks/obj_type_2
[INFO][2024/04/29 03:30:06 PM] Loading LBEP/obj_type_2
[INFO][2024/04/29 03:30:08 PM] Loading objects/obj_type_2 (37477, 5) (37477 filtered: None)
[INFO][2024/04/29 03:30:14 PM] Loading segmentation (51, 2304, 2304)
[INFO][2024/04/29 03:30:14 PM] Closing HDF file: /Volumes/lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_SPC_PDPN_ZO1/sc_analyses/20230801_20X_23-03-011B6_DAPI_SP-C_PDPN_ZO-1_Multichannel Z-Stack_20230801_1440_iat2.h5



 20230801_20X_23-03-011B6_DAPI_SP-C_PDPN_ZO-1_Multichannel Z-Stack_20230801_1440_iat2.h5
Number of channel 1 positive cells: 2532
Number of channel 1 negative cells: 719
Percentage of positive cells: 77.88%


/Users/dayn/miniforge3/envs/loc/lib/python3.12/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):



 20230801_20X_23-03-011B6_DAPI_SP-C_PDPN_ZO-1_Multichannel Z-Stack_20230801_1440_iat2.h5
Number of channel 2 positive cells: 3110
Number of channel 2 negative cells: 141
Percentage of positive cells: 95.66%


...

/Users/dayn/miniforge3/envs/loc/lib/python3.12/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
[INFO][2024/04/29 03:35:06 PM] Opening HDF file: /Volumes/lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_NKX21_PDPN_ZO1/sc_analyses/20230801_20X_23-03-011B5_DAPI_NKX2-1_PDPN_ZO-1_Multichannel Z-Stack_20230801_1435_iat2.h5...
[INFO][2024/04/29 03:35:06 PM] Loading tracks/obj_type_2
[INFO][2024/04/29 03:35:07 PM] Loading LBEP/obj_type_2
[INFO][2024/04/29 03:35:09 PM] Loading objects/obj_type_2 (50760, 5) (50760 filtered: None)
[INFO][2024/04/29 03:35:43 PM] Loading segmentation (61, 2304, 2304)
[INFO][2024/04/29 03:35:43 PM] Closing HDF file: /Volumes/lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_NKX21_PDPN_ZO1/sc_a


 20230801_20X_23-03-011B5_DAPI_NKX2-1_PDPN_ZO-1_Multichannel Z-Stack_20230801_1435_iat2.h5
Number of channel 1 positive cells: 578
Number of channel 1 negative cells: 2379
Percentage of positive cells: 19.55%


/Users/dayn/miniforge3/envs/loc/lib/python3.12/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):



 20230801_20X_23-03-011B5_DAPI_NKX2-1_PDPN_ZO-1_Multichannel Z-Stack_20230801_1435_iat2.h5
Number of channel 2 positive cells: 2509
Number of channel 2 negative cells: 448
Percentage of positive cells: 84.85%


/Users/dayn/miniforge3/envs/loc/lib/python3.12/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
[INFO][2024/04/29 03:36:28 PM] Opening HDF file: /Volumes/lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_experiments/DAPI_NKX21_PDPN_ZO1/sc_analyses/20230801_20X_23-03-002A5_DAPI_NKX201_PDPN_ZO-1_Multichannel Z-Stack_20230801_1441_iat2.h5...
[INFO][2024/04/29 03:36:30 PM] Loading tracks/obj_type_2
[INFO][2024/04/29 03:36:32 PM] Loading LBEP/obj_type_2


# Save out as df/excel

In [ ]:
### define parameters to plot in table
cols = ['Cell ID', f'Max. int. {channels(0).name}', f'Max. int. {channels(1).name}', f'Max. int. {channels(2).name}', f'Max. int. {channels(3).name}', 'Average X', 'Average Y']
### extract some intial measurements
df = pd.DataFrame([(track.ID, np.nanmax(track['mean_intensity-0']),np.nanmax(track['mean_intensity-1']),np.nanmax(track['mean_intensity-2']),np.nanmax(track['mean_intensity-3']), np.mean(track.x), np.mean(track.y)) for track in tracks], columns= cols)